# PROYECTO FINAL DE CIENCIA DE DATOS - BOOTCAMP CÓDIGO FACILITO

## Realizado por Gonzalo Mahserdjian y Simón Maquilón

### INSTALAR REQUERIMIENTOS

Instalo los requerimientos necesarios para este Jupiter Notebook:

In [ ]:
!python -m pip install python-dotenv polars

In [ ]:
!pip freeze > requirements.txt

### OBTENCIÓN DE DATOS PARA DATASETS

Para armar el DataSet con los datos con los que voy a trabajar, realicé una gran consulta de SQL (Big Query), como traer todos los años juntos a veces se colgaba la conexión por VPN o traía menos datos, los separé primero por años y luego los unifiqué en un único arhivo:

In [ ]:
import csv
import pyodbc
import asyncio
import polars as pl

from pathlib import Path
from dotenv import load_dotenv, dotenv_values

load_dotenv(dotenv_path=".env", verbose=True)
cfg = dotenv_values(".env")

DATASET_SYNC = False
PRINT_QUERY = False
START_YEAR = 2018
END_YEAR = 2023
DATA_PATH = 'data/'
SQL_FILE_PATH = 'sql/bigquery.sql'

def read_sql_file(path: str, search: str, replace: str) -> None:
    """
    Edits file content
    """
    if Path(path).suffix == '.sql':
        try:
            with open(path, 'r', encoding='utf-8') as file:
                data = file.read()

            return data.replace(search, replace)

        except FileNotFoundError as error:
            print(error)
            return None

def get_query(year: str) -> str:
    return read_sql_file(SQL_FILE_PATH, '{year}', year)


if __name__ == '__main__':

    try:
        db_string = f"""DRIVER={cfg['DB_DRIVER']};
                        SERVER={cfg['DB_HOST']};
                        DATABASE={cfg['DB_DATABASE']};
                        UID={cfg['DB_USER']};
                        PWD={cfg['DB_PASSWORD']}"""

        if DATASET_SYNC:
            connect = pyodbc.connect(db_string)

            for year in range(START_YEAR, END_YEAR+1):
                if(PRINT_QUERY):
                    print(get_query(str(year)))

                with connect.cursor() as cursor:
                    cursor.execute(get_query(str(year)))
                    rows = cursor.fetchall()

                    with open(r'./'+DATA_PATH+'/DataSet_'+str(year)+'.csv', 'w', encoding="utf-8", newline='') as csvfile:
                        writer = csv.writer(csvfile)
                        writer.writerow([x[0] for x in cursor.description])  # column headers
                        
                        for row in rows:
                            writer.writerow(row)

            await asyncio.sleep(10)

            df = pl.read_csv(f'{DATA_PATH}DataSet_*.csv')
            df.write_csv(f'{DATA_PATH}DataSet.csv')
        
            print('>>> ¡Dataset sincronizado con éxito!')
        else:
            print('>>> ¡Sincronización de Dataset no habilitada, se utilizará la versión offline!')

    except pyodbc.OperationalError as err:
        print('No fue posible realizar la conexión!')
        print(err)
         

### LECTURA DEL CSV Y DEPURACIÓN DEL DATASET

Variables para el DataSet unificado con los últimos 6 años y para el DataSet limpio:

In [ ]:
file_path = f'{DATA_PATH}DataSet.csv'
file_path_cleaned = f'{DATA_PATH}DataSetCleaned.csv'

Leo el DataSet unificado, y visualizo las primeras 5 columnas:

In [ ]:
df = pl.read_csv(file_path)
df.head()

Exploro las las métricas básicas sobre las columnas del DataSet:

In [ ]:
df.describe()

Dada la característica del DataSet que armé y a donde apunto con los mismos mucho no me sirven estos resultados, por empezar las primeras 2 columnas son IDs de clientes y sus contratos, tengo strings de nombres de barrios y constantes de nombres de planes, periodos, etc, lo único rescatable sería que "1 de Mayo" es el barrio que menor cantidad de veces sale y su opuesto "Villa Eucaristica" es el que más sale, y por otro lado que 6mbps es el plan más chico (en 2018) de internet (es un plan viejo, actualmente 50mbps es el menor) y 300mbps es el mayor, más adelante voy a comprobar todo esto.

Verifico la cantidad de líneas y columnas del DataSet:

In [ ]:
df.shape

A continuación verifico si hay valores NA provenientes desde la consulta SQL:

In [ ]:
df.select(pl.lit('NA').str.contains(pl.col('Internet_Tipo')))

In [ ]:
df.select(pl.lit('NA').str.contains(pl.col('Estado')))

Busco si hay valores nulos:

In [ ]:
df.select(pl.all().null_count()).to_dicts()[0]

Se encontró un único valor nulo en columna "Barrio", realizo la limpieza...

In [ ]:
df = df.fill_null(value='Sin barrio')

Busco duplicados en la columna de "ID_Cliente":

In [ ]:
df.filter(pl.count("ID_Cliente").over(df.columns) > 1)

Unifico los duplicados de "ID_Cliente" manteniendo los primeros resultados de nada caso, como la consulta SQL ordena los más recientes primero, quedarían los contratos vigentes:

In [ ]:
df = df.unique(keep='first', maintain_order=True)

Realizo la misma búsqueda de duplicados en la columna de "ID_Contrato":

In [ ]:
df.filter(pl.count("ID_Contrato").over(df.columns) > 1)

Unifico ahora los duplicados de "ID Contrato":

In [ ]:
df = df.unique(keep='first', maintain_order=True)

Con el DataSet limpio, guardo todo en el archivo "DataSetCleaned.csv":

In [ ]:
df.write_csv(file_path_cleaned)

Leo el nuevo DataSet y vuelvo a verificar si quedó algún nulo y duplicados:

In [ ]:
df = pl.read_csv(file_path_cleaned)
df.select(pl.all().null_count())

In [ ]:
df.select(pl.all().null_count()).to_dicts()[0]

In [ ]:
df.filter(pl.count("ID_Contrato").over(df.columns) > 1)

Visualizo parcialmente el DataSet:

In [ ]:
print(df)

### ANÁLISIS DE LOS DATOS OBTENIDOS

Luego de realizada la consulta con la big query al motor de MSSQL, y posterior limpieza de los datos con Polars completando 1 solo dato nulo de un barrio inexistente en la celda "Barrio" del cliente en cuestión y eliminando los duplicados a favor del más reciente, se verifica que los datos son consistentes poorque todas las lineas (rows) tienen la misma cantidad de valores en las columnas (cols).

¿Cúal es el por qué de los duplicados?: La base de datos es de un sistema donde no estaba pensado para evaluar los upgrades de los planes de internet, entonces logré obtener ese dato comparando tablas con contratos que tengan algún ticket con leyenda "Upgrade" (mejora en el plan del servicio de internet) en el descriptor del mismo, con un margen de 12 meses y esto en algun que otro cliente me trajo esos duplicados. Para generalizar, es porque en algunos sufrieron mudanzas, otros se arrepintieron del servicio o bien le ofrecieron pasarse a fibra óptica y luego por imposibilidades técnicas u otras que no puedo comentar, aún esa zona no contará con tendido de fibra óptica, dejando el ticket en "Time out" y devolviendo al cliente solamente con el servicio de Aire o Inalámbrico.

Como el gran trabajo se realizó en la big query y luego la limpieza, no se detectan anomalías. Bueno, creo que si podría citar una anomalía que me encontre al realizar la big query, en donde habían clientes que no entraban en la categoría de "Upgrade" pero no eran ni "Altas" ni "Bajas" del servicio, entonces los bauticé como "Fail Upgrade", porque fueron casos donde imposibilidades técnicas hicieron que no se concrete el Upgrade pero que no se perdía el cliente, quedaba con el mismo plan ya existente y sin cambios, o sea tampoco se subscribió al servicio de TV por IP.

A continuacíon se evaluarán los datos para identificar tendencias evidentes, y responder a las siguientes preguntas:
* Altas del servicio de internet SIPA-AIRE vs SIPA-FIBRA en los últimos 6 años.
* Bajas del servicio de internet SIPA-AIRE vs SIPA-FIBRA en los últimos 6 años.
* Mudanzas del servicio de internet SIPA AIRE a SIPA-FIBRA.
* Ver que barrio es el que tiene mayor demanda en el tiempo.
* Ver que ancho de banda es el que tiene mayor demanda en el tiempo.
* Ver que plan de internet es el que tiene mayor demanda en el tiempo.
* Ver que cantidad de clientes tienen contratado IPTV en el tiempo.